In [3]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

DATA_PATH = '/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/'

DNA_SEQUENCE_PATH = pathlib.Path(DATA_PATH).joinpath('capstone_dictionary_gene_fasta_early_adult_IMPC_genes_flat_v3.pkl')

EFFECT_SIZE_PATH = pathlib.Path(DATA_PATH).joinpath('capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023.pkl')

RESULT_PATH = pathlib.Path(DATA_PATH).joinpath('effect_size_protein_sequence.pkl')

In [4]:
with open(DNA_SEQUENCE_PATH, 'rb') as f:
    data = pickle.load(f)

ModuleNotFoundError: No module named 'Bio'

In [3]:
records = []
results = data['results']
for result in results:
    primary_accession = result['primaryAccession']
    uniprotkb_id = result['uniProtkbId']
    if 'sequence' in result:
        sequence = result['sequence']['value']
        if 'genes' in result and len(result['genes']) != 0:
            for gene in result['genes']:
                gene_names = []
                if 'geneName' in gene:
                    gene_names.append(gene['geneName'])
                if 'synonyms' in gene:
                    gene_names.extend(gene['synonyms'])
                if 'orfNames' in gene:
                    gene_names.extend(gene['orfNames'])
                for gene_name in gene_names:
                    if 'value' in gene_name:
                        gene_symbol = gene_name['value']
                        records.append({
                            'gene_symbol': gene_symbol,
                            'primary_accession': primary_accession,
                            'uniprotkb_id': uniprotkb_id,
                            'sequence': sequence
                        })

In [4]:
df = pd.DataFrame(records)

df['gene_symbol_harmonized'] = df['gene_symbol'].str.strip().str.lower()

df_gene_symbol_protein_sequence = df.copy()

In [5]:
df = pd.read_pickle(EFFECT_SIZE_PATH)

df['gene_symbol_harmonized'] = df['gene_symbol'].str.strip().str.lower()

df = df.merge(df_gene_symbol_protein_sequence, how='left', on='gene_symbol_harmonized')

df = df.drop(['gene_symbol_harmonized'], axis=1)

df.to_pickle(RESULT_PATH)

df

,procedure_name,parameter_name,allele_accession_id,gene_accession_id,gene_symbol_x,allele_symbol,zygosity,strain,est_f_ea,se_f_ea,p_f_ea,p_adj_f_ea,est_m_ea,se_m_ea,p_m_ea,p_adj_m_ea,est_type_ea,add_info_ea,wt_f_ea_count,wt_m_ea_count,mut_f_ea_count,mut_m_ea_count,total_number_of_mutants,gene_symbol_y,primary_accession,uniprotkb_id,sequence
0,body composition (dexa lean/fat),body weight,mgi:5694170,MGI:1913300,0610009B22Rik,0610009B22Rik<tm1b(EUCOMM)Hmgu>,homozygote,c57bl/6n,-0.56979,0.194647,0.003428,0.013711,-0.56979,0.194647,0.003428,0.00457,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes included"", ""multibatch_in_analysis"":""Data contains multi batches"", ""summary_statistics"":{""Female_+/+"":{""count"":5034, ""mean"":-0.66090496104691, ""sd"":0.641908237274802},""Male_+/+"":{""count"":5020, ""mean"":0.664501504988317, ""sd"":0.84284064821203},""Female_Mutant"":{""count"":7, ""mean"":-1.0758261056034, ""sd"":0.273554080125102},""Male_Mutant"":{""count"":8, ""mean"":-0.15889979899848, ""sd"":0.890373886377904}},""variability"":0.0239348495381865,""formula"":""data_point ~ Genotype + Sex"",",5034.0,5020.0,7.0,8.0,15.0,0610009B22Rik,Q8R3W2,Q8R3W2_MOUSE,MSGSFYFVIVGHHDNPVFEMEFLPAGKAESKDEHRHLNQFIAHAALDLVDENMWLSNNMYLKTVDKFNEWFVSAFVTAGHMRLIMLHDVRHEDGIKNFFTDVYDLYIKFAMNPFYEPNSPIRSSAFERKVQFLGKKHLLN
1,body weight,body weight,not-released-0541dc0af7,MGI:1923511,0610040J01Rik,0610040J01Rik<em1(IMPC)Ccpcz>,homozygote,c57bl/6ncrl,-0.956256,0.558026,0.086929,0.115905,-0.956256,0.558026,0.086929,0.115905,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes included"", ""multibatch_in_analysis"":""Data contains multi batches"", ""summary_statistics"":{""Female_+/+"":{""count"":5083, ""mean"":-0.66719727645237, ""sd"":0.614496001786977},""Male_+/+"":{""count"":5198, ""mean"":0.655009374033684, ""sd"":0.862357881283796},""Female_Mutant"":{""count"":8, ""mean"":-1.34729676555856, ""sd"":0.488772124516029},""Male_Mutant"":{""count"":8, ""mean"":-0.324574486903232, ""sd"":0.688857900835301}},""variability"":0.818393706904924,""formula"":""data_point ~ Genotype + Sex"",",5083.0,5198.0,8.0,8.0,16.0,0610040J01Rik,A0A0G2JDR8,A0A0G2JDR8_MOUSE,MGCRCCKMIQSYLFDPVQVPSPGFVNEVNNCKLEEDDTVRLKGTQNSEVEVPRNALHDGSLSNSESRGSTTGLPHQGPLPQEDSEERPCVEKQGIVNGISPTATLQSVRSSRLHQVDNSSWASSPWVATIDSAHLAQPFLEGEDYRKQSCLLPTLEGTQMVGHGDCRAPAEALAVADHIPYIPAPDYPQLWSPTVDNADPEEKDCLFENHSEVEP
2,body weight,body weight,not-released-0541dc0af7,MGI:1923511,0610040J01Rik,0610040J01Rik<em1(IMPC)Ccpcz>,homozygote,c57bl/6ncrl,-0.956256,0.558026,0.086929,0.115905,-0.956256,0.558026,0.086929,0.115905,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes included"", ""multibatch_in_analysis"":""Data contains multi batches"", ""summary_statistics"":{""Female_+/+"":{""count"":5083, ""mean"":-0.66719727645237, ""sd"":0.614496001786977},""Male_+/+"":{""count"":5198, ""mean"":0.655009374033684, ""sd"":0.862357881283796},""Female_Mutant"":{""count"":8, ""mean"":-1.34729676555856, ""sd"":0.488772124516029},""Male_Mutant"":{""count"":8, ""mean"":-0.324574486903232, ""sd"":0.688857900835301}},""variability"":0.818393706904924,""formula"":""data_point ~ Genotype + Sex"",",5083.0,5198.0,8.0,8.0,16.0,0610040J01Rik,A0A0G2JDX0,A0A0G2JDX0_MOUSE,MGCRCCKMIQSYLFDPVQVPSPGFVNEVNNCKLEEDDTVRLKGTQNSEVEVPRNALHDGSLSNSESRGSTTGLPHQGPLPQEDSEERPCVEKQGIVNGISPTATLQSVRSSRLHQVDNSSWASSPWVATIDSAHLAQPFLEGEDYRKQSCLLPTLEGTQMVGHGDCRAPAEALAVADHIPYIPAPDYPQLWSPTVDNADPEE
3,body weight,body weight,not-released-0541dc0af7,MGI:1923511,0610040J01Rik,0610040J01Rik<em1(IMPC)Ccpcz>,homozygote,c57bl/6ncrl,-0.956256,0.558026,0.086929,0.115905,-0.956256,0.558026,0.086929,0.115905,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes included"", ""multibatch_in_analysis"":""Data contains multi batches"", ""summary_statistics"":{""Female_+/+"":{""count"":5083, ""mean"":-0.66719727645237, ""sd"":0.614496001786977},""Male_+/+"":{""count"":5198, ""mean"":0.655009374033684, "